# Practical Exam: Grocery Store Sales

FoodYum is a grocery store chain that is based in the United States.

Food Yum sells items such as produce, meat, dairy, baked goods, snacks, and other household food staples.

As food costs rise, FoodYum wants to make sure it keeps stocking products in all categories that cover a range of prices to ensure they have stock for a broad range of customers. 

## Data

The data is available in the table `products`.

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with 2022. |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

# Task 1

Last year (2022) there was a bug in the product system. For some products that were added in that year, the `year_added` value was not set in the data. As the year the product was added may have an impact on the price of the product, this is important information to have. 

Write a query to determine how many products have the `year_added` value missing. Your output should be a single column, `missing_year`, with a single row giving the number of missing values.

In [42]:
-- Write your query for task 1 in this cell
SELECT count(*) as missing_year FROM products
WHERE year_added IS NULL;


# Task 2

Given what you know about the year added data, you need to make sure all of the data is clean before you start your analysis. The table below shows what the data should look like. 

Write a query to ensure the product data matches the description provided. Do not update the original table.  

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with last year (2022). |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

In [43]:
-- Write your query for task 2 in this cell
SELECT
        CAST(product_id AS INTEGER) AS product_id
		-- check the 5 product_type
       ,Cast( CASE WHEN product_type IN ('Bakery','Dairy','Meat','Produce','Snacks') THEN product_type ::TEXT
		ELSE 'Unknown'
		END AS TEXT ) product_type
		-- check the 7 brands 
	 ,CASE WHEN brand IN (SELECT DISTINCT brand FROM public.products) THEN
        COALESCE(NULLIF(brand, '-'), 'Unknown')::TEXT
		ELSE 'Unknown'::TEXT
		END AS brand
	-- ROUND to 2 dp,remove grams in the data,change to median
	   ,ROUND(
    	COALESCE(
        CAST(REPLACE(weight, ' grams', '') AS DECIMAL),
        (SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY CAST(REPLACE(weight, ' grams', '') AS DECIMAL)) FROM public.products)
    	)::DECIMAL, 2
		) AS weight
-- add $ , round to 2 dp,change to median
	    ,ROUND(
    	COALESCE(
        CAST(price AS DECIMAL),
        (SELECT PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY CAST(price AS DECIMAL)) FROM public.products)
    	), 2
		) AS price
			-- change to integer, and replace with 0
, COALESCE(CAST(average_units_sold AS INTEGER), 0) AS average_units_sold
-- replace with 2022
, COALESCE(CAST(year_added AS INTEGER), 2022) AS year_added
--  uppercase, a,b,c,d , Repplace with Unknown
		, UPPER(
    	CASE
        WHEN stock_location IN ('A', 'B', 'C', 'D') THEN stock_location
        WHEN stock_location IN ('a', 'b', 'c', 'd') THEN UPPER(stock_location)
        ELSE 'Unknown'
    	END
		)::TEXT AS stock_location
	FROM public.products
	where product_id IS NOT NULL

# Task 3

To find out how the range varies for each product type, your manager has asked you to determine the minimum and maximum values for each product type.   

Write a query to return the `product_type`, `min_price` and `max_price` columns. 

In [44]:
-- Write your query for task 3 in this cell
SELECT product_type, min(price) as min_price, max(price) as max_price
FROM clean_data
GROUP BY product_type

# Task 4

The team want to look in more detail at meat and dairy products where the average units sold was greater than ten. 

Write a query to return the `product_id`, `price` and `average_units_sold` of the rows of interest to the team. 

In [45]:
-- Write your query for task 4 in this cell
SELECT product_id, price, average_units_sold
FROM clean_data
WHERE product_type IN ('Meat', 'Dairy') AND average_units_sold >10 ;